In [34]:
import requests
from datetime import datetime
from typing import Iterable
from tqdm import tqdm
import time
import re
from functools import reduce
from bs4 import BeautifulSoup
from multiprocessing.pool import ThreadPool

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [35]:
def get_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    driver = webdriver.Chrome(executable_path = '../../bins/chromedriver', options = options)
    return driver
#https://chromedriver.storage.googleapis.com/95.0.4638.69/chromedriver_mac64_m1.zip
#https://chromedriver.storage.googleapis.com/95.0.4638.69/chromedriver_linux64.zip

In [12]:
get_driver()

/var/folders/lm/flgwr_ks0cn6xv9bcn3f_d2m0000gn/T/ipykernel_58014/2455803570.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path = '../../bins/chromedriver', options = options)


<selenium.webdriver.chrome.webdriver.WebDriver (session="9f1c1a4b3ae866ced9e79edba4a10623")>

In [61]:
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--no-sandbox')
#options.add_argument("--whitelisted-ips")

capabilities = webdriver.DesiredCapabilities.CHROME
capabilities['browserVersion'] = '95.0'
driver = webdriver.Remote('http://localhost:4445', desired_capabilities=capabilities, options = options)

/var/folders/lm/flgwr_ks0cn6xv9bcn3f_d2m0000gn/T/ipykernel_58014/3432947372.py:8: DeprecationWarning: desired_capabilities has been deprecated, please pass in an Options object with options kwarg
  driver = webdriver.Remote('http://localhost:4445', desired_capabilities=capabilities, options = options)


WebDriverException: Message: Could not start a new session. Error while creating session with the driver service. Stopping driver service: Could not start a new session. Response code 500. Message: unknown error: Chrome failed to start: crashed.
  (chrome not reachable)
  (The process started from chrome location /usr/bin/google-chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Build info: version: '4.1.0', revision: 'aea69daa1c'
System info: host: 'chrome', ip: '192.168.32.2', os.name: 'Linux', os.arch: 'amd64', os.version: '5.10.47-linuxkit', java.version: '11.0.11'
Driver info: driver.version: unknown
Build info: version: '4.1.0', revision: 'aea69daa1c'
System info: host: 'chrome', ip: '192.168.32.2', os.name: 'Linux', os.arch: 'amd64', os.version: '5.10.47-linuxkit', java.version: '11.0.11'
Driver info: driver.version: unknown
Stacktrace:
    at org.openqa.selenium.grid.node.config.DriverServiceSessionFactory.apply (DriverServiceSessionFactory.java:

In [ ]:
driver

In [423]:
import psycopg2
from configparser import ConfigParser


def config(filename='/Users/alexander/match-prediction/database.ini', section='postgresql'):
    parser = ConfigParser()
    parser.read(filename)

    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

def insert_games(data):
    try:
        params = config()
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        cur = conn.cursor()
        args_str = ','.join(cur.mogrify(f"({', '.join(['%s']*len(x))})", list(x.values())).decode("utf-8") for x in data)
        
        cur.execute("INSERT INTO games VALUES " + args_str) 
        conn.commit()
        cur.close()
        print(f"{len(data)} rows've been inserted")
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()

def retrieve(condition: str = None):
    try:
        params = config()
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params)

        cur = conn.cursor()
        if condition:
            cur.execute(f"SELECT * FROM games WHERE {condition}") 
        else:
            cur.execute("SELECT * FROM games") 
        records = cur.fetchall()

        cur.close()
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if conn is not None:
            conn.close()
    return records

In [415]:
def get_paths(driver):
    match_paths = list(map(lambda x: x.get_dom_attribute('href'), 
                           driver.find_elements(By.CLASS_NAME, 'ind_match_wrapper')))
    match_paths = match_paths[1:]
    match_paths = list(map(lambda x: 'https://www.xscores.com/' + x, match_paths))
    teams = list(map(lambda x: x.split('/')[-3].split('-vs-'), match_paths))
    return match_paths, teams

In [416]:
def get_match_stats(driver, path):
    def get_ftr(score_1, score_2):
        if score_1 > score_2:
            return 'H'
        elif score_1 < score_2:
            return 'A'
        else:
            return 'D'
        
    try:       
        driver.get(path)

        regex_ref = re.compile("Referee: ((\w+|\s+)+)\n")
        regex_stat = lambda x: re.compile(f"{x}\n(\d+)\n(\d+)")

        stats_dict = dict()
        stat_attrs = [
            ('Goal Attempts', 'hs', 'as'), 
            ('Fouls Commited', 'hf', 'af'), 
            ('Corners', 'hc', 'ac'),
            ('Yellow Cards', 'hy', 'ay'), 
            ('Red Cards', 'hr', 'ar')
        ]

        score = driver.find_element(By.CLASS_NAME, 'match_details_score').text.split(' - ')
        date = driver.find_element(By.CLASS_NAME, 'match_details_date').text
        date = datetime.strptime(date, 'Match date: %d-%m-%Y / %H:%M')
        try:
            referee = regex_ref.search(driver.find_element(By.CLASS_NAME, 'game_info_overview').text).group(1)
            bet1x, betx, bet2x = driver.find_element(By.CLASS_NAME, 'mDetails-odds').text.split('\n')[1::2]
        except:
            referee, bet1x, betx, bet2x = None, None, None, None

        stats_dict['Date'] = date
        stats_dict['referee'] = referee
        stats_dict['B365H'] = bet1x
        stats_dict['B365D'] = betx
        stats_dict['B365A'] = bet2x
        stats_dict['played'] = 0

        stats_dict['fthg'] = None
        stats_dict['ftag'] = None
        stats_dict['ftr'] = None
        for stat_name, h_stat, a_stat in stat_attrs:
            stats_dict[h_stat] = None
            stats_dict[a_stat] = None

        #if match has been played
        if len(score) != 1:
            stats_dict['played'] = 1
            stats_dict['fthg'] = score[0]
            stats_dict['ftag'] = score[1]
            stats_dict['ftr'] = get_ftr(score[0], score[1])

            driver.find_element(By.XPATH, '//*[@id="scoretable"]/div/div[6]/div/div/div/div[4]').click()
            start_time = time.time()
            stats = ''
            while stats == '':
                stats = driver.find_element(By.CLASS_NAME, 'match_info_wrapper').text
                time.sleep(0.5)
                if time.time() - start_time > 10:
                    raise TimeoutError

            for stat_name, h_stat, a_stat in stat_attrs:
                match_info = regex_stat(stat_name).search(stats)
                stats_dict[h_stat] = match_info.group(1)
                stats_dict[a_stat] = match_info.group(2)
    except Exception as e:
        print(e)

    return stats_dict

In [417]:
def max_rounds(path):
    driver = get_driver()
    driver.get(path)
    round_select = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'round_select')))
    max_rounds = int(round_select.text.split('\n')[-1])
    return max_rounds

def get_round_links(path: str, rounds_n: Iterable):
    try:
        driver = get_driver()
        driver.get(path)
        round_selection = lambda x: (
            WebDriverWait(driver, 5)
            .until(EC.presence_of_element_located((By.CLASS_NAME, 'round_select')))
            .find_element(By.XPATH, f'//*[@id="scoretable"]/div[1]/div[3]/div/div/div[1]/div/select/option[{x}]')
            .click()
        )

        match_paths, pair_teams = [], []
        for n in tqdm(rounds_n, desc = 'getting round links'):
            WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.CLASS_NAME, 'round_select')))
            round_selection(n)
            time.sleep(1)
            paths, teams = get_paths(driver)
            match_paths.extend(paths)
            pair_teams.extend(teams)
    except Exception as e:
        print(e)
    finally:
        driver.close()
        driver.quit()
    return match_paths, pair_teams

In [432]:
def get_matches_stats(path, rounds_n: Iterable):
    stats_data = []
    driver = get_driver()
    range(1, max_rounds(path)+1)
    links, matches = get_round_links(path, rounds_n)
    
    for link in tqdm(links, desc = 'getting stats about matches'):
        stats_data.append(get_match_stats(driver, link))
        
    for stat_dict, teams in zip(stats_data, matches):
        stat_dict['HomeTeam'] = teams[0]
        stat_dict['AwayTeam'] = teams[1]
        
    driver.close()
    driver.quit()
    return stats_data

In [433]:
path = 'https://www.xscores.com/soccer/england/premier-league'
stats = get_matches_stats(path)

/var/folders/lm/flgwr_ks0cn6xv9bcn3f_d2m0000gn/T/ipykernel_2383/1996232620.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../bins/chromedriver', options = options)
getting stats about matches: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:22<00:00,  1.32it/s]


In [450]:
condition = """
referee IS NOT NULL
and bet_1x IS NOT NULL
and played = 0
"""
dat = retrieve(condition)

Connecting to the PostgreSQL database...


In [451]:
dat

[(datetime.datetime(2021, 11, 20, 14, 30),
  'Paul Tierney',
  5.0,
  3.5,
  1.72,
  0,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'leicester',
  'chelsea'),
 (datetime.datetime(2021, 11, 20, 17, 0),
  'Martin Atkinson',
  3.6,
  3.5,
  1.95,
  0,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'norwich',
  'southampton'),
 (datetime.datetime(2021, 11, 20, 17, 0),
  'Robert Jones',
  2.62,
  3.25,
  2.7,
  0,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'newcastle',
  'brentford'),
 (datetime.datetime(2021, 11, 20, 17, 0),
  'Simon Hooper',
  2.87,
  3.1,
  2.55,
  0,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  'burnley',
  'crystal-palace'),
 (datetime.datetime(2021, 11, 20, 17, 0),
  'Anthony Taylor',
  2.5,
  3.25,
  2.8,
  0,
  None,
  None,


In [434]:
insert_games(stats)

Connecting to the PostgreSQL database...
30 rows've been inserted


In [ ]:
#add referee to the table
#do not forget to change code in scripts